<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/DepthAnyVideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Depth ANy Video:<br>
https://github.com/Nightmare-n/DepthAnyVideo

In [1]:
!git clone https://github.com/Nightmare-n/DepthAnyVideo
%cd DepthAnyVideo

Cloning into 'DepthAnyVideo'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 62 (delta 15), reused 4 (delta 4), pack-reused 38 (from 1)
Receiving objects: 100% (62/62), 14.68 MiB | 17.24 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/content/DepthAnyVideo


In [1]:
%cd DepthAnyVideo

/content/DepthAnyVideo


In [3]:
!python run_infer.py --data_path ./demos/arch_2.jpg --output_dir ./outputs/ --max_resolution 320

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
2024-12-30 16:41:21.858084: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-30 16:41:21.877915: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-30 16:41:21.883745: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-30 16:41:21.898624: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimiz

In [ ]:
!python run_infer.py --data_path ./demos/wooly_mammoth.mp4 --output_dir ./outputs/ --max_resolution 64

In [3]:
import argparse
import logging
import os
import random

from easydict import EasyDict
import numpy as np
import torch
from dav.pipelines import DAVPipeline
from dav.models import UNetSpatioTemporalRopeConditionModel
from diffusers import AutoencoderKLTemporalDecoder, FlowMatchEulerDiscreteScheduler
from dav.utils import img_utils


def seed_all(seed: int = 0):
    """
    Set random seeds of all components.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


if "__main__" == __name__:
    logging.basicConfig(level=logging.INFO)

    parser = argparse.ArgumentParser(
        description="Run video depth estimation using Depth Any Video."
    )

    parser.add_argument(
        "--model_base",
        type=str,
        default="hhyangcs/depth-any-video",
        help="Checkpoint path or hub name.",
    )

    # data setting
    parser.add_argument(
        "--data_path", type=str, required=False, help="input data directory."
    )

    parser.add_argument(
        "--output_dir", type=str, required=False, help="Output directory."
    )

    # inference setting
    parser.add_argument(
        "--denoise_steps",
        type=int,
        default=3,
        help="Denoising steps, 1-3 steps work fine.",
    )
    parser.add_argument(
        "--num_frames",
        type=int,
        default=32,
        help="Number of frames to infer per forward",
    )
    parser.add_argument(
        "--decode_chunk_size",
        type=int,
        default=16,
        help="Number of frames to decode per forward",
    )
    parser.add_argument(
        "--num_interp_frames",
        type=int,
        default=16,
        help="Number of frames for inpaint inference",
    )
    parser.add_argument(
        "--num_overlap_frames",
        type=int,
        default=6,
        help="Number of frames to overlap between windows",
    )
    parser.add_argument(
        "--max_resolution",
        type=int,
        default=1024,  # decrease for faster inference and lower memory usage
        help="Maximum resolution for inference.",
    )

    parser.add_argument("--seed", type=int, default=None, help="Random seed.")

    args = parser.parse_args([])
    args.data_path = "./demos/wooly_mammoth.mp4"
    args.output_dir = "./outputs/"
    args.max_resolution = 64
    cfg = EasyDict(vars(args))

    if cfg.seed is None:
        import time

        cfg.seed = int(time.time())
    seed_all(cfg.seed)

    device_type = "cuda"
    device = torch.device(device_type)

    os.makedirs(cfg.output_dir, exist_ok=True)
    logging.info(f"output dir = {cfg.output_dir}")

    vae = AutoencoderKLTemporalDecoder.from_pretrained(cfg.model_base, subfolder="vae")
    scheduler = FlowMatchEulerDiscreteScheduler.from_pretrained(
        cfg.model_base, subfolder="scheduler"
    )
    vae.half()

    unet = UNetSpatioTemporalRopeConditionModel.from_pretrained(
        cfg.model_base, subfolder="unet"
    )
    unet.half()

    unet_interp = UNetSpatioTemporalRopeConditionModel.from_pretrained(
        cfg.model_base, subfolder="unet_interp"
    )
    unet_interp.half()


    pipe = DAVPipeline(
        vae=vae,
        unet=unet,
        unet_interp=unet_interp,
        scheduler=scheduler,
    )
    pipe = pipe.to(device)

    file_name = cfg.data_path.split("/")[-1].split(".")[0]
    is_video = cfg.data_path.endswith(".mp4")
    if is_video:
        num_interp_frames = cfg.num_interp_frames
        num_overlap_frames = cfg.num_overlap_frames
        num_frames = cfg.num_frames
        assert num_frames % 2 == 0, "num_frames should be even."
        assert (
            2 <= num_overlap_frames <= (num_interp_frames + 2 + 1) // 2
        ), "Invalid frame overlap."
        max_frames = (num_interp_frames + 2 - num_overlap_frames) * (num_frames // 2)
        image, fps = img_utils.read_video(cfg.data_path, max_frames=max_frames)
    else:
        image = img_utils.read_image(cfg.data_path)

    image = img_utils.imresize_max(image, cfg.max_resolution)
    image = img_utils.imcrop_multi(image)
    image_tensor = np.ascontiguousarray(
        [_img.transpose(2, 0, 1) / 255.0 for _img in image]
    )
    image_tensor = torch.from_numpy(image_tensor).to(device)
    print('inputs:', image_tensor.shape)
    print('cfg.num_overlap_frames:', cfg.num_overlap_frames)
    print('cfg.num_interp_frames:', cfg.num_interp_frames)
    print('cfg.decode_chunk_size:', cfg.decode_chunk_size)
    print('cfg.denoise_steps:', cfg.denoise_steps)

    with torch.no_grad(), torch.autocast(device_type=device_type, dtype=torch.float16):
        pipe_out = pipe(
            image_tensor,
            num_frames=cfg.num_frames,
            num_overlap_frames=cfg.num_overlap_frames,
            num_interp_frames=cfg.num_interp_frames,
            decode_chunk_size=cfg.decode_chunk_size,
            num_inference_steps=cfg.denoise_steps,
        )

    disparity = pipe_out.disparity
    disparity_colored = pipe_out.disparity_colored
    image = pipe_out.image
    print('out_depth:', image.shape)
    # (N, H, 2 * W, 3)
    merged = np.concatenate(
        [
            image,
            disparity_colored,
        ],
        axis=2,
    )
    print('out_depth merged:', merged.shape)

    if is_video:
        img_utils.write_video(
            os.path.join(cfg.output_dir, f"{file_name}.mp4"),
            merged,
            fps,
        )
    else:
        img_utils.write_image(
            os.path.join(cfg.output_dir, f"{file_name}.png"),
            merged[0],
        )


inputs: torch.Size([192, 3, 32, 64])
cfg.num_overlap_frames: 6
cfg.num_interp_frames: 16
cfg.decode_chunk_size: 16
cfg.denoise_steps: 3
rgb_latent: torch.Size([1, 30, 4, 4, 8])
latent_model_input: torch.Size([1, 30, 8, 4, 8])
latent_model_input: torch.Size([1, 30, 8, 4, 8])
latent_model_input: torch.Size([1, 30, 8, 4, 8])


100%|██████████| 15/15 [00:06<00:00,  2.25it/s]


out_depth: (184, 32, 64, 3)


In [4]:
print('out_depth merged:', merged.shape)

out_depth merged: (184, 32, 128, 3)


In [7]:
192/6

32.0